In [ ]:
import sys
import json
sys.path.append("../")
from src.data_utils import load_captions, make_train_test_images
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_hub as hub


In [ ]:
# Loading data
CAPTIONS_PATH = "../data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"
IMAGES_PATH = "../data/Flickr_Data/Images/"
captions = load_captions(CAPTIONS_PATH)
train, test, train_images, test_images = make_train_test_images(CAPTIONS_PATH, IMAGES_PATH)

In [ ]:
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

detector = hub.load(module_handle).signatures['default']

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
img = load_img(train_images[0])
converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

In [ ]:
out = detector(converted_img)

In [ ]:
plt.imshow(img)

In [ ]:
out['detection_boxes']

In [ ]:
def extract_regions(img, bounding_boxes):
    width, height, _ = img.shape
    features = []
    for n, (y1, x1, y2, x2) in enumerate(bounding_boxes):
        x1 = int(width * x1)
        x2 = int(width * x2)
        y1 = int(height * y1)
        y2 = int(height * y2)
        if (x1 != x2) and (y1 != y2):
            features.append(img[x1:x2+1, y1:y2+1, :])
    return features        

In [ ]:
features = extract_regions(converted_img[0], out['detection_boxes'])

In [ ]:
len(features)

In [ ]:
plt.imshow(features[9])